In [1]:
import nbimporter
import Application as ap

import unittest
from unittest.mock import patch, MagicMock
import pandas as pd

In [2]:
class TestRestaurantDataProcessor(unittest.TestCase):
    def setUp(self):
        """
        Prepare resources for tests.
        """
        self.url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
        self.country_code_file = "country_codes.csv"
        self.output_file_path = "processed_restaurants.csv"
        self.processor = ap.RestaurantDataProcessor(self.url, self.country_code_file, self.output_file_path)

    def test_fetch_json_data(self):
        """
        Test fetching JSON data from an API.
        """
        with patch('requests.get') as mock_get:
            # Mocking a successful API response
            mock_response = MagicMock()
            mock_response.ok = True
            mock_response.json.return_value = {"restaurants": []}
            mock_get.return_value = mock_response

            # Testing successful data fetch
            result = self.processor.fetch_json_data()
            self.assertIsNotNone(result)
            mock_get.assert_called_once_with(self.url)

            # Mocking an unsuccessful API response and testing failure handling
            mock_response.ok = False
            result = self.processor.fetch_json_data()
            self.assertIsNone(result)

    def test_read_country_code_data(self):
        """
        Test reading country code data from a CSV file.
        """
        with patch('pandas.read_csv') as mock_read_csv:
            # Mocking successful CSV file read
            mock_read_csv.return_value = pd.DataFrame(data={"Country Code": [1], "Country": ["India"]})
            result = self.processor.read_country_code_data()
            self.assertIsNotNone(result)
            mock_read_csv.assert_called_once_with(self.country_code_file)

            # Mocking FileNotFoundError to test error handling
            mock_read_csv.side_effect = FileNotFoundError
            result = self.processor.read_country_code_data()
            self.assertIsNone(result)

    def test_merge_data(self):
        """
        Test merging JSON data with CSV country code data.
        """
        parsed_data = {"restaurants": [{"restaurant": {"location": {"country_id": 1}}}]}
        country_code_df = pd.DataFrame(data={"Country Code": [1], "Country": ["India"]})
        result = self.processor.merge_data(parsed_data, country_code_df)
        self.assertIn("Country", result.columns)  # Check if 'Country' column is in the merged DataFrame

    def test_process_merged_data(self):
        """
        Test processing of the merged DataFrame.
        """
        merged_df = pd.DataFrame(data={
            "restaurant.R.res_id": [1],
            "restaurant.name": ["Test Restaurant"],
            "Country": ["India"],
            "restaurant.location.city": ["Test City"],
            "restaurant.user_rating.votes": [100],
            "restaurant.user_rating.aggregate_rating": ["4.5"],
            "restaurant.cuisines": ["Indian"]
        })
        result = self.processor.process_merged_data(merged_df)
        self.assertIn("Restaurant Name", result.columns)  # Checking for column renaming
        self.assertTrue(result["User Aggregate Rating"].dtype == float)  # Checking data type conversion

    def test_export_df_to_csv(self):
        """
        Test exporting a DataFrame to a CSV file.
        """
        with patch('pandas.DataFrame.to_csv') as mock_to_csv:
            df = pd.DataFrame()
            self.processor.export_df_to_csv(df)
            mock_to_csv.assert_called_once_with(self.output_file_path, index=False)

    def test_run(self):
        """
        Test the complete data processing workflow.
        """
        # Mocking all methods called within the run method
        with patch.object(self.processor, 'fetch_json_data') as mock_fetch, \
             patch.object(self.processor, 'read_country_code_data') as mock_read, \
             patch.object(self.processor, 'merge_data') as mock_merge, \
             patch.object(self.processor, 'process_merged_data') as mock_process, \
             patch.object(self.processor, 'export_df_to_csv') as mock_export:
                
            # Setting return values for mocked methods
            mock_fetch.return_value = {"restaurants": []}
            mock_read.return_value = pd.DataFrame()
            mock_merge.return_value = pd.DataFrame()
            mock_process.return_value = pd.DataFrame()

            self.processor.run()

            # Verifying that each method was called once
            mock_fetch.assert_called_once()
            mock_read.assert_called_once()
            mock_merge.assert_called_once()
            mock_process.assert_called_once()
            mock_export.assert_called_once()

In [7]:
class TestEventDataProcessor(unittest.TestCase):
    def setUp(self):
        """
        Prepare resources for each test.
        """
        self.dummy_df = pd.DataFrame({
            "restaurant.zomato_events": [[{"event": {"event_id": 1}}]]
        })
        self.processor = ap.EventDataProcessor(self.dummy_df, 2021, 12, "output.csv")

    def test_expand_and_normalize_events(self):
        """
        Test expanding and normalizing events data.
        """
        # Patch the DataFrame's explode method and the json_normalize function
        with patch('pandas.DataFrame.explode') as mock_explode, \
             patch('pandas.json_normalize') as mock_json_normalize:
            mock_explode.return_value = self.dummy_df  # Mock the result of explode
            mock_json_normalize.return_value = pd.DataFrame({"event.event_id": [1]})  # Mock the result of json_normalize

            result_df = self.processor.expand_and_normalize_events()

            # Verify that explode and json_normalize were called as expected
            mock_explode.assert_called_once_with("restaurant.zomato_events")
            mock_json_normalize.assert_called_once()
            # Ensure the resulting DataFrame has the expected column from normalization
            self.assertIn("event.event_id", result_df.columns)

    def test_filter_events_by_date(self):
        """
        Test filtering events by date.
        """
        
        # Setup a mock DataFrame to filter
        events_df = pd.DataFrame({
            "event.start_date": ["2019-03-01", "2019-05-01"],
            "event.end_date": ["2019-05-31", "2020-01-01"]
        })

        filtered_df = self.processor.filter_events_by_date(events_df, 2019, 4)

        expected_df = pd.DataFrame({
            "event.start_date": ["2019-03-01"],
            "event.end_date": ["2019-03-01"]
        })

        pd.testing.assert_frame_equal(filtered_df.reset_index(drop=True), expected_df)

    def test_process_events_data(self):
        """
        Test processing of events data.
        """
        # Patch the internal methods used in the process_events_data method
        with patch.object(self.processor, 'expand_and_normalize_events', return_value=pd.DataFrame()) as mock_expand, \
             patch.object(self.processor, 'filter_events_by_date', return_value=pd.DataFrame()) as mock_filter:

            self.processor.process_events_data()

            # Verify that each method was called once as part of the data processing workflow
            mock_expand.assert_called_once()
            mock_filter.assert_called_once()

    def test_export_events_to_csv(self):
        """
        Test exporting events data to CSV.
        """
        # Patch the to_csv method of DataFrame
        with patch('pandas.DataFrame.to_csv') as mock_to_csv:
            df_to_export = pd.DataFrame({"event.event_id": [1]})

            self.processor.export_events_to_csv(df_to_export, "output.csv")

            # Verify that to_csv was called with the correct arguments
            mock_to_csv.assert_called_once_with("output.csv", index=False)

    def test_run(self):
        """
        Test the full run method orchestrating the events data processing.
        """
        # Patch the methods called within the run method
        with patch.object(self.processor, 'process_events_data', return_value=pd.DataFrame()) as mock_process, \
             patch.object(self.processor, 'export_events_to_csv') as mock_export:

            self.processor.run()

            # Verify that the process and export methods are each called once
            mock_process.assert_called_once()
            mock_export.assert_called_once()

In [4]:
class TestRatingStatisticsProcessor(unittest.TestCase):
    def setUp(self):
        """
        Prepare resources for each test.
        
        Initializes a RatingStatisticsProcessor instance with a dummy DataFrame and a test output path.
        """
        self.dummy_df = pd.DataFrame({
            "restaurant.user_rating.rating_text": ["Excellent", "Good", "Average"],
            "restaurant.user_rating.aggregate_rating": ["4.5", "3.5", "2.5"]
        })
        self.output_file_path = "test_output.json"
        self.processor = ap.RatingStatisticsProcessor(self.dummy_df, self.output_file_path)

    def test_filter_and_convert_ratings(self):
        """
        Test filtering by specified rating texts and conversion of rating to float.
        
        Ensures that the DataFrame is correctly filtered and ratings are converted to float.
        """
        specified_texts =  ["Excellent","Very Good", "Good", "Average", "Poor"]
        result_df = self.processor.filter_and_convert_ratings(specified_texts)

        # Verify that only rows with specified rating texts are present
        self.assertTrue(all(result_df['restaurant.user_rating.rating_text'].isin(specified_texts)))
        # Check if ratings are converted to float
        self.assertTrue(pd.api.types.is_float_dtype(result_df['restaurant.user_rating.aggregate_rating']))

    def test_analyze_rating_distribution(self):
        """
        Test analysis of rating distribution.
        
        Checks that the method returns a DataFrame with correct aggregation of ratings.
        """
        filtered_df = self.processor.filter_and_convert_ratings(["Excellent", "Good"])
        result_statistics = self.processor.analyze_rating_distribution(filtered_df)

        # Verify the structure and content of the result DataFrame
        self.assertIn("min", result_statistics.columns)
        self.assertIn("max", result_statistics.columns)

    def test_export_to_json(self):
        """
        Test exporting rating statistics to a JSON file.
        
        Verifies that the method attempts to write to the specified file path.
        """
        # Mock DataFrame to test export
        rating_statistics = pd.DataFrame({
            "User Rating": ["Excellent","Very Good", "Good", "Average", "Poor"],
            "min": [4.0, 3.0,2.0,1.0,0.0],
            "max": [5.0, 4.0, 3.0,2.0,1.0]
        })

        with patch('pandas.DataFrame.to_json') as mock_to_json:
            self.processor.export_to_json(rating_statistics)

            # Verify that to_json was called with the correct file path
            mock_to_json.assert_called_once_with(self.output_file_path, orient='records')

    def test_run(self):
        """
        Test the full run method orchestrating the rating statistics processing.
        
        Verifies that the process correctly filters, analyzes, and exports rating data.
        """
        # Patch the internal methods to isolate the run method's workflow
        with patch.object(self.processor, 'filter_and_convert_ratings', return_value=pd.DataFrame()) as mock_filter, \
             patch.object(self.processor, 'analyze_rating_distribution', return_value=pd.DataFrame()) as mock_analyze, \
             patch.object(self.processor, 'export_to_json') as mock_export:

            self.processor.run(["Excellent","Very Good", "Good", "Average", "Poor"])

            # Verify that each method in the workflow is called once
            mock_filter.assert_called_once()
            mock_analyze.assert_called_once()
            mock_export.assert_called_once()


In [5]:
loader = unittest.TestLoader()
suite = unittest.TestSuite()

# Add tests to the test suite
suite.addTests(loader.loadTestsFromTestCase(TestEventDataProcessor))
# Run the test suite
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_expand_and_normalize_events (__main__.TestEventDataProcessor.test_expand_and_normalize_events)
Test expanding and normalizing events data. ... ok
test_export_events_to_csv (__main__.TestEventDataProcessor.test_export_events_to_csv)
Test exporting events data to CSV. ... ok
test_process_events_data (__main__.TestEventDataProcessor.test_process_events_data)
Test processing of events data. ... ERROR
test_run (__main__.TestEventDataProcessor.test_run)
Test the full run method orchestrating the events data processing. ... ok

ERROR: test_process_events_data (__main__.TestEventDataProcessor.test_process_events_data)
Test processing of events data.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\junke\AppData\Local\Temp\ipykernel_26608\1689022256.py", line 55, in test_process_events_data
    self.processor.process_events_data()
  File "Application.ipynb", line 70, in process_events_data
  File "c:\Users\junke\AppD

Events data exported successfully to output.csv


<unittest.runner.TextTestResult run=4 errors=1 failures=0>

In [6]:
# Load tests from the TestStringMethods class
